# Model Creation: Chicken Detector

### 1. Importing Modules

In [ ]:
!pip install -q torchmetrics

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import transforms

from torchmetrics import Accuracy

In [ ]:
# Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### 2. Functionizing Training/Testing

In [ ]:
accuracy = Accuracy(task="binary").to(device) # defining accuracy metric

In [ ]:
# Train Step
def train_step(self,
              model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              optimizer: torch.optim.Optimizer,
              accuracy: Accuracy,
              device: torch.device=device # set default device
              ):
  """
  Train Step Function:
  Trains a PyTorch model on binary image classification on specified training dataset.
  Prints the average train loss and train accuracy across all batches.
  """
  train_loss, train_acc = 0, 0

  # Device Agnostic Code
  model.to(device)
  accuracy.to(device)

  model.train() # put model into training mode

  # Loop through training batches
  for batch, (image, label) in enumerate(dataloader):
    image, label = image.to(device), label.to(device) # Put data on target device

    # 1. Forward Pass
    y_pred = model(image)
    # 2. Calculate and accumulate loss/accuracy
    loss = loss_fn(y_pred, label)
    train_loss += loss.item()
    train_acc += accuracy(y_pred, label).item()
    # 3. Optimizer zero grad
    optimizer.zero_grad()
    # 4. Loss backwards
    loss.backward()
    # 5. Optimizer step
    optimizer.step()

  # Calculate average loss/accuracy across all batches
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  # Print out Train Loss and Accuracy
  print(f"Train Loss: {train_loss:.3f} | Train Accuracy: {train_acc*100:.3f}")

In [ ]:
# Test Step
def test_step(self,
              model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy: Accuracy,
              device = torch.device=device
              ):
  """
  Test Step Function:
  Tests a PyTorch model agains the training dataset to evaluate its preformance.
  Prints the average test loss and test accuracy across all batches.
  """
  test_loss, test_acc = 0, 0

  # Device Agnostic Code
  model.to(device)
  accuracy.to(device)

  model.eval() # put model into eval mode

  with torch.inference_mode(): # test with inference mode, deactivates unnecessary features
    for batch, (image, label) in enumerate(dataloader):
      image, label = image.to(device), label.to(device) # Put data on target device

      # 1. Forward Pass
      y_pred = model(image)
      # 2. Calculate the Loss/Accuracy
      test_loss += loss_fn(y_pred, label).item()
      test_acc += accuracy(y_pred, label).item()

    # Calculate the averages across all batches
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    # Print out average test loss and accuracy
    print(f"Test Loss: {test_loss:.3f} | Test Accuracy: {test_acc*100:.3f}")



In [ ]:
placeholder_data = torch.rand(size=(32, 3, 224, 384)) # placeholder data to experiment with before loading custom data